In [1]:
import numpy as np
from mindspore import Tensor, ops
import mindspore as ms
import sys
sys.path.append('..')
from data_processing import X_train,X_test,y_train,y_test
from DQAS_tool import Mindspore_ansatz2,sampling_from_structure,nmf_gradient,vag_nnp2,zeroslike_grad_nnp,generate_pauli_string
import numpy as np
import json
import tensorcircuit as tc
import tensorflow as tf
from DQAS_tool import best_from_structure,DQAS_accuracy
# #Operator Pool


/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/simulator/__init__.py:17: UserWarning: Unable import mqvector gpu backend due to: cannot import name '_mq_vector_gpu' from partially initialized module 'mindquantum' (most likely due to a circular import) (/opt/miniconda3/envs/MindSpore/lib/python3.9/site-packages/mindquantum/__init__.py)
  from .available_simulator import SUPPORTED_SIMULATOR
Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [2]:

unbound_opeartor_pool = \
["X0 X1 I2 I3 I4 I5 I6 I7", 
 "I0 I1 X2 X3 I4 I5 I6 I7",
 "I0 I1 I2 I3 X4 X5 I6 I7",
 "I0 I1 I2 I3 I4 I5 X6 X7",
 "Y0 Y1 I2 I3 I4 I5 I6 I7",
 "I0 I1 Y2 Y3 I4 I5 I6 I7",
 "I0 I1 I2 I3 Y4 Y5 I6 I7",
 "I0 I1 I2 I3 I4 I5 Y6 Y7",
 "Z0 Z1 I2 I3 I4 I5 I6 I7", 
 "I0 I1 Z2 Z3 I4 I5 I6 I7",
 "I0 I1 I2 I3 Z4 Z5 I6 I7",
 "I0 I1 I2 I3 I4 I5 Z6 Z7",
 
 ]
bound_opeartor_pool = \
['ZYIXXXZX',
 'IZZYYXZI',
 'IZZYIXZI',
 'ZZYYYYYY']

num_layer = 12
# 定义标准差和形状
stddev = 0.02
shape_parametized = len(unbound_opeartor_pool) #12
shape_unparametized = len(bound_opeartor_pool) #6
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)

nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp)

# np.random.seed(10)
# unbound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[0] for i in range(shape_parametized)]
# bound_opeartor_pool = [generate_pauli_string(n=8,seed=i)[1] for i in range(shape_parametized,shape_parametized+shape_unparametized)]

In [5]:
def best_from_structure(structures: np.array)->Tensor:
    prob = ops.Softmax()(ms.Tensor(structures))
    return ops.Argmax(axis=-1)(prob)

In [3]:

#from DQAS_tool import  DQASAnsatz_from_result,DQAS_accuracy
K = tc.set_backend("tensorflow")
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.05))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
verbose = True
# 设置超参数
epochs = 30
batch_size=10
shape_nnp = (num_layer, shape_parametized)
shape_stp = (num_layer, shape_parametized+shape_unparametized)
nnp = np.random.normal(loc=0.0, scale=stddev, size=shape_nnp).astype(np.float64)
stp = np.random.normal(loc=0.0, scale=stddev, size=shape_stp).astype(np.float64)
print(stp.shape)
avcost1 = 0

ops_onehot = ops.OneHot(axis=-1)

batch_loss_history=[] # 记录每个epoch的batch_size损失值
structure_distribution_history=[] # 记录每个epoch的结构参数
ansatz_params_history=[] # 记录每个epoch的网络参数
best_candidates_history=[] # 记录每个epoch的最佳候选
acc_history = [] #记录每个epoch的准确率

 
for epoch in range(20):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    tmp = np.stack([sampling_from_structure(stp,num_layer,shape_parametized) for _ in range(batch_size)])
    batch_structure = ops_onehot(ms.Tensor(tmp),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
    #print(batch_structure.shape)
    # print(tmp,batch_structure)
    loss_value = []
    grad_nnps = []
    grad_stps = []
    
    for i in batch_structure:
        #print(ops.Argmax()(i))          
        infd, grad_nnp = vag_nnp2(Structure_params=i,
                                  Ansatz_params=nnp,
                                  paramerterized_pool=unbound_opeartor_pool,unparamerterized_pool=bound_opeartor_pool,
                                  num_layer=num_layer,n_qbits=8)(ms.Tensor(X_train),ms.Tensor(y_train))
        
        grad_nnp_zeroslike = zeroslike_grad_nnp(batch_sturcture=i,grad_nnp=grad_nnp[0],shape_parametized=shape_parametized,ansatz_parameters=nnp)
        gs = nmf_gradient(structures=stp,oh=i,num_layer=num_layer,size_pool=stp.shape[1])
        #print(infd,grad_nnp)
        loss_value.append(infd)
        grad_nnps.append(ms.Tensor(grad_nnp_zeroslike,dtype=ms.float64))
        grad_stps.append(gs)

      
    infd = ops.stack(loss_value)
    gnnp = ops.addn(grad_nnps)
    gstp = [(infd[i] - avcost2) * grad_stps[i] for i in range(infd.shape[0])]
    gstp_averge = ops.addn(gstp) / infd.shape[0]
    avcost1 = sum(infd) / infd.shape[0]
    # print(f'loss={infd}\ngrad_nnp={gnnp}\ngrandient_stp={gstp_averge}')
    
    gnnp_tf = tf.convert_to_tensor(gnnp.asnumpy(),dtype=tf.float64)
    nnp_tf = tf.convert_to_tensor(nnp,dtype=tf.float64)
    gstp_averge_tf = tf.convert_to_tensor(gstp_averge.reshape(stp.shape).asnumpy(),dtype=tf.float64)
    stp_tf = tf.convert_to_tensor(stp,dtype=tf.float64)
     # 更新参数
    nnp_tf = network_opt.update(gnnp_tf, nnp_tf)
    stp_tf = structure_opt.update(gstp_averge_tf, stp_tf) 
    
    nnp = nnp_tf.numpy()
    stp = stp_tf.numpy()

    batch_loss_history.append(avcost1)
    structure_distribution_history.append(stp)
    ansatz_params_history.append(nnp)
    #best_candidates_history.append(best_from_structure(cand_preset.asnumpy()))
    cand_preset = best_from_structure(stp)
    best_candidates_history.append(cand_preset.asnumpy())
    
    if len(best_candidates_history) > 5 and np.array_equal(best_candidates_history[-1], best_candidates_history[-2]) and np.array_equal(best_candidates_history[-2], best_candidates_history[-3]):
        print('满足停止条件!')
        break
    
    if epoch % 2 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched平均损失: ",
            avcost1,
        )
    
        if verbose:
            print(
                "strcuture parameter: \n",
                stp,
                "\n network parameter: \n",
                nnp,
            )
        
        print("最好的候选结构:",cand_preset)
        stp_for_test = ops_onehot(ms.Tensor(cand_preset),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
        test_ansatz = Mindspore_ansatz2(Structure_p=stp_for_test,parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)
        
        ansatz_parameters=[]
        for layerIndex,i in enumerate(cand_preset):
            if i >=len(unbound_opeartor_pool):
                continue
            else:
                ansatz_parameters.append(nnp[layerIndex,i])
        
        acc = DQAS_accuracy(ansatz=test_ansatz,Network_params=ansatz_parameters,n_qbits=8)
        acc_history.append(acc)
        print(f'二分类准确率 Acc ={acc*100}% ')
        
        

(12, 16)
----------epoch 0-----------
batched平均损失:  0.86629117
strcuture parameter: 
 [[-0.07498237  0.0563186   0.0409926  -0.07371429 -0.07440935  0.08301378
   0.05439937  0.07163363 -0.04565407 -0.03784892  0.03306379 -0.05703985
  -0.04575074  0.08273636  0.06947167 -0.065     ]
 [-0.06632764  0.05549198 -0.06060009  0.06014769 -0.05280241 -0.09892874
  -0.02347103  0.03902388 -0.07434002  0.06812599 -0.063189   -0.06689808
   0.04062509 -0.04026065  0.04132347 -0.05684164]
 [ 0.04780725  0.0695007   0.0192473   0.04670219 -0.02544048 -0.02586304
  -0.05482795 -0.03995679 -0.06478362 -0.05589477  0.08371726 -0.05489396
   0.0293811  -0.07104557  0.04484598  0.04966036]
 [ 0.05500215 -0.03922897 -0.00454273 -0.00849402  0.06530166  0.03962056
  -0.05421289  0.06723703  0.06109192 -0.02067519  0.04281464 -0.04912282
  -0.0157415  -0.04190949  0.04318574  0.02889312]
 [ 0.03914627 -0.05144199 -0.07557504  0.06254124  0.06955264 -0.00895589
   0.07355601 -0.06963142  0.01695348 -0.077

In [4]:
best_candidates_history[-1]

array([13,  3,  0,  5, 13, 11,  5,  5, 15,  4, 14,  0], dtype=int32)

In [5]:
ops_onehot = ops.OneHot(axis=-1)
stp_onehot = ops_onehot(ms.Tensor(best_candidates_history[-1]),shape_parametized+shape_unparametized,ms.Tensor(1),ms.Tensor(0))
ansatz = Mindspore_ansatz2(Structure_p=stp_onehot,parameterized_pool=unbound_opeartor_pool,unparameterized_pool=bound_opeartor_pool,num_layer=num_layer,n_qbits=8)

In [6]:
ansatz

┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┏━━━━━━━━━━━━┓                                            
q0: ──┨ H ┠─┨ RY(encoder0) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃                                            
q1: ──┨ H ┠─┨ RY(encoder1) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃                                            
q2: ──┨ H ┠─┨ RY(encoder2) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃                                            
q3: ──┨ H ┠─┨ RY(encoder3) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃ 1*IZZYYXZI ┃                                            
q4: ──┨ H ┠─┨ RY(encoder4) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃                                            
q5: ──┨ H ┠─┨ RY(encoder5) ┠─┨            ┠─────────────────────────────────────[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃                                            
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃ ┏━━━┓                                      
q6: ──┨ H ┠─┨ RY(encoder6) ┠─┨            ┠─┨ H ┠───■───────────────────────■───[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┃            ┃ ┗━━━┛   ┃                       ┃          
      ┏━━━┓ ┏━━━━━━━━━━━━━━┓ ┃            ┃ ┏━━━┓ ┏━┻━┓ ┏━━━━━━━━━━━━━━━┓ ┏━┻━┓        
q7: ──┨ H ┠─┨ RY(encoder7) ┠─┨            ┠─┨ H ┠─┨╺╋╸┠─┨ RZ(2*ansatz0) ┠─┨╺╋╸┠─[red bold]↯[/]─     
      ┗━━━┛ ┗━━━━━━━━━━━━━━┛ ┗━━━━━━━━━━━━┛ ┗━━━┛ ┗━━━┛ ┗━━━━━━━━━━━━━━━┛ ┗━━━┛        
                                                                                       
      ┏━━━┓                               ┏━━━┓                                        
q0: ──┨ H ┠───■───────────────────────■───┨ H ┠─────────────────────────────────────[red bold]↯[/]─ 
      ┗━━━┛   ┃                       ┃   ┗━━━┛                                        
      ┏━━━┓ ┏━┻━┓ ┏━━━━━━━━━━━━━━━┓ ┏━┻━┓ ┏━━━┓                                        
q1: ──┨ H ┠─┨╺╋╸┠─┨ RZ(2*ansatz1) ┠─┨╺╋╸┠─┨ H ┠─────────────────────────────────────[red bold]↯[/]─ 
      ┗━━━┛ ┗━━━┛ ┗━━━━━━━━━━━━━━━┛ ┗━━━┛ ┗━━━┛                                        
                                          ┏━━━━━━━━━┓                                  
q2: ──────────────────────────────────────┨ RX(π/2) ┠───■───────────────────────■───[red bold]↯[/]─ 
                                          ┗━━━━━━━━━┛   ┃                       ┃      
                                          ┏━━━━━━━━━┓ ┏━┻━┓ ┏━━━━━━━━━━━━━━━┓ ┏━┻━┓    
q3: ──────────────────────────────────────┨ RX(π/2) ┠─┨╺╋╸┠─┨ RZ(2*ansatz2) ┠─┨╺╋╸┠─[red bold]↯[/]─ 
                                          ┗━━━━━━━━━┛ ┗━━━┛ ┗━━━━━━━━━━━━━━━┛ ┗━━━┛    
                                                                                       
q4: ────────────────────────────────────────────────────────────────────────────────[red bold]↯[/]─ 
                                                                                       
                                                                                       
q5: ────────────────────────────────────────────────────────────────────────────────[red bold]↯[/]─ 
                                                                                       
      ┏━━━┓                             

In [10]:
stp_onehot[4,:]

Tensor(shape=[16], dtype=Int64, value= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [11]:
bound_opeartor_pool

['ZYIXXXZX', 'IZZYYXZI', 'IZZYIXZI', 'ZZYYYYYY']

In [7]:
from Test_tool import Test_ansatz

In [8]:
result = Test_ansatz(ansatz=ansatz,learning_rate=0.01,epochs=20)

epoch: 1 step: 20, loss is 0.6235052347183228
epoch: 1 step: 40, loss is 0.7312543392181396
epoch: 2 step: 11, loss is 0.7184774875640869
epoch: 2 step: 31, loss is 0.5083945393562317
epoch: 3 step: 2, loss is 0.7274727821350098
epoch: 3 step: 22, loss is 0.5547995567321777
epoch: 3 step: 42, loss is 0.4975627362728119
epoch: 4 step: 13, loss is 0.4385916590690613
epoch: 4 step: 33, loss is 0.4785218834877014
epoch: 5 step: 4, loss is 0.4734545648097992
epoch: 5 step: 24, loss is 0.6104376316070557
epoch: 5 step: 44, loss is 0.6520686745643616
epoch: 6 step: 15, loss is 0.5395877361297607
epoch: 6 step: 35, loss is 0.5470375418663025
epoch: 7 step: 6, loss is 0.5569216012954712
epoch: 7 step: 26, loss is 0.5568155646324158
epoch: 7 step: 46, loss is 0.5374386310577393
epoch: 8 step: 17, loss is 0.4200115203857422
epoch: 8 step: 37, loss is 0.5145796537399292
epoch: 9 step: 8, loss is 0.759008526802063
epoch: 9 step: 28, loss is 0.6292473077774048
epoch: 9 step: 48, loss is 0.5193205475